Chapter 3 of D2L Textbook


In [55]:
import d2l
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, data as gdata, nn

import time
import random
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### 3.1.5. Exercises

1. **Assume that we have some data  $x_1,… x_n \in \mathbb{R}$ . Our goal is to find a constant  𝑏  such that $\sum_i (x_i - b)^2$  is minimized.**

    a.**Find the optimal closed form solution.**
    
    b.**What does this mean in terms of the Normal distribution?**
    
$$\frac{\partial}{\partial b}{\sum_i (x_i - b)^2} = - 2 \sum_i (x_i - b) = 0  \textbf{  =>  } \hat{b} = \bar{x} = mean(x_i)$$
$$\frac{\partial}{\partial b}{\sum_i (x_i - b)^2} = 2 > 0  \textbf{  =>  } \hat{b} \text{ is a minimum.  }$$

   This equals to the mean ($\mu$) of the normal distribution.


 
2. **Assume that we want to solve the optimization problem for linear regression with quadratic loss explicitly in closed form. To keep things simple, you can omit the bias  𝑏  from the problem.**

    a. **Rewrite the problem in matrix and vector notation (hint - treat all the data as a single matrix).**

    $$L(\mathbf{w}) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\mathbf{w}^\top \mathbf{x}^{(i)} - y^{(i)}\right)^2. = ||\mathbf{X}\mathbf{w} - \mathbf{y}||^2$$
    
    b. **Compute the gradient of the optimization problem with respect to  𝑤 .**
    
    $$\frac{\partial}{\partial \mathbf{w}}{L(\mathbf{w})} = 2 X^T (X\mathbf{w} - \mathbf{y})$$
    
    c. **Find the closed form solution by solving a matrix equation.**
    
    $$\mathbf{\hat{w}} = (X^TX)^{-1} X^T \mathbf{y}$$
    
    d. **When might this be better than using stochastic gradient descent (i.e. the incremental optimization approach that we discussed above)? When will this break (hint - what happens for high-dimensional  𝑥 , what if many observations are very similar)?.**
    
    When the features are linear independent.
    
    For high demensional x, if some of the features are of high correlations, then $(X^T X)^{-1}$ may not existed.
    
    
3. **Assume that the noise model governing the additive noise  𝜖  is the exponential distribution. That is,  $p(\epsilon)=\frac{1}{2}exp(−|\epsilon|)$.**

    a.**Write out the negative log-likelihood of the data under the model  $−log p(Y|X)$.**
    
    $$−log p(Y|X) = -log(\frac{1}{2} exp(-|X\mathbf{w}-Y|)) = |X\mathbf{w}-Y| - log(\frac{1}{2}) $$
    
    b. **Can you find a closed form solution?**
    
    $$ \mathbf{w} = argmin_{\mathbf{w}} |X\mathbf{w}-Y| $$
    
    c. **Suggest a stochastic gradient descent algorithm to solve this problem. What could possibly go wrong (hint - what happens near the stationary point as we keep on updating the parameters). Can you fix this?**
    
    
4. **Compare the runtime of the two methods of adding two vectors using other packages (such as NumPy) or other programming languages (such as MATLAB).**



#### 3.2.9. Exercises

1. **What would happen if we were to initialize the weights 𝐰=0 . Would the algorithm still work?**

    It doesn't work. Each neuron at a layer will have exactly same gradient at each update, hence all the neurons will learn the same thing.
    

2. **Assume that you’re Georg Simon Ohm trying to come up with a model between voltage and current. Can you use autograd to learn the parameters of your model.**




In [53]:
lr = 0.1  # Learning rate
num_epochs = 10  # Number of iterations
batch_size = 10
num_examples = 1000

## construct voltage, current & resistance
resistance = 10 # nd.random.normal(scale=0.01, shape=(num_examples, 1))
current = nd.random.normal(scale=1, shape=(num_examples, 1))
voltage = resistance * current
print(voltage.shape)
voltage += nd.random.normal(scale=0.01, shape=(num_examples, 1))

## initialize parameters                            
loss = gloss.L2Loss()
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.1))
trainer_w = gluon.Trainer(net.collect_params('.*weight'), 'sgd', {'learning_rate': lr, 'wd': 1})
trainer_b = gluon.Trainer(net.collect_params('.*bias'), 'sgd', {'learning_rate': lr})
                            
for epoch in range(num_epochs):
    b = nd.zeros(shape=(1))
    w = nd.random.normal(scale=0.01, shape=(1))
    for X, y in data_iter(batch_size, current, voltage):
        with autograd.record():
            l = loss(net(X), y)  # Minibatch loss
#         import ipdb
#         ipdb.set_trace()
        l.backward()  # Compute gradient on l with respect to [w,b]
        trainer_w.step(batch_size)
        trainer_b.step(batch_size)
    train_l = loss(linreg(voltage, w, b), current)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

(1000, 1)
epoch 1, loss 0.477460
epoch 2, loss 0.597413
epoch 3, loss 0.385625
epoch 4, loss 0.644568
epoch 5, loss 0.315570
epoch 6, loss 0.455128
epoch 7, loss 0.450721
epoch 8, loss 0.501864
epoch 9, loss 0.325878
epoch 10, loss 0.545352


3. **Can you use Planck’s Law to determine the temperature of an object using spectral energy density.**

    ??????
    

4. **What are the problems you might encounter if you wanted to extend autograd to second derivatives? How would you fix them?**

    The second order gradient (Jacobian matrix) will have $n^2$ more parameters for a layer with n neuron weights.
    

5. **Why is the reshape function needed in the squared_loss function?**

    Since the prediction tensor can have arbitrary shape, while its size(the product of all the dimension times together) is equal to true label "y", hence we just need to reshape true label "y" as same as the shape of prediction y.
    

6. **Experiment using different learning rates to find out how fast the loss function value drops.**

    A good start choose of lr is 0.1. Sometimes 1 is too high and the model cannot converge, but 0.01 is too small and the model converge slowly.
    

7. **If the number of examples cannot be divided by the batch size, what happens to the data_iter function’s behavior?**

    It will take the last few examples which are less than a batchsize to train.

In [50]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i: min(i + batch_size, num_examples)])
        yield features.take(j), labels.take(j)

#### 3.3.9. Exercises


1. **If we replace l = loss(output, y) with l = loss(output, y).mean(), we need to change trainer.step(batch_size) to trainer.step(1) accordingly. Why?**

    From the documentation of `Gluon.Trainer.step`, gradient will be normalized by 1/batch_size, which is the same as take a mean with batch_size 1.
    
    See details: https://mxnet.apache.org/api/python/gluon/gluon.html#mxnet.gluon.Trainer


2. **Review the MXNet documentation to see what loss functions and initialization methods are provided in the modules gluon.loss and init. Replace the loss by Huber’s loss.**

    L1Loss, L2Loss, SigmoidBinaryCrossEntropyLoss, SoftmaxCrossEntropyLoss, KLDivLoss, etc.
    https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.Loss
    

3. **How do you access the gradient of dense.weight?**

    `with autograd.record():` records computation history on the fly to calculate gradients later.
    
    To compute gradient with respect to an NDArray x, first call `x.attach_grad()` to allocate space for the gradient. Then, start a with `autograd.record():` block, and do some computation. Finally, call `backward()` on the result. See details https://mxnet.apache.org/api/python/autograd/autograd.html


In [54]:
x = mx.nd.array([1,2,3,4])
x.attach_grad()
with mx.autograd.record():
    y = x * x + 1
y.backward()
print(x.grad)


[2. 4. 6. 8.]
<NDArray 4 @cpu(0)>


#### 3.4.6. Exercises

1. **Show that the Kullback-Leibler divergence  $𝐷(𝑝‖𝑞)$  is nonnegative for all distributions  𝑝  and  𝑞 . Hint - use Jensen’s inequality, i.e. use the fact that  $−log𝑥$  is a convex function.**

    Since $-logx$ is convex, then

    $$D(p\|q) = -\sum_j p(j) \log q(j) - H[p] = \sum_j p(j) \log \frac{p(j)}{q(j)} = \sum_j - p(j) \log \frac{q(j)}{p(j)} \geq  -\log (\sum_j p(j) \frac{q(j)}{p(j)}) = 0$$

2. **Show that  $ log \sum j \exp(o_j)$  is a convex function in o.**

    For any j > 1, $f_j (x) = x^j$ is a convex function. hence, 
    $$f_j (\sum_o o_j) \leq \sum_o f_j(o_j) => (\sum_o o_j)^j \leq \sum_o (o_j)^j$$
    
    $$i.e. \log j \exp(\sum_o o_j) \leq \sum_o log j \exp o_j$$
    
    Hence, $ log \sum j \exp(o_j)$  is a convex function in o.
    
    
3. **We can explore the connection between exponential families and the softmax in some more depth。**

    a. **Compute the second derivative of the cross entropy loss  𝑙(𝑦,𝑦̂)  for the softmax.**
    
    By definition of softmax, 
    $$softmax(y, \hat{y}) = -\sum_j y_j \log \hat{y}_j = \sum_j y_j \log \sum_k \exp(o_k) - \sum_j y_j o_j = \log \sum_k \exp(o_k) - \sum_j y_j o_j$$
    
    Hence, the derivative is
    $$\partial_{o_j} softmax(y, \hat{y}) = \frac{\exp(o_j)}{\sum_k \exp(o_k)} - y_j = \mathrm{softmax}(o_j) - y_j = \hat{y_j} - y_j = \Pr(y = j|x) - y_j$$
    
    b. **Compute the variance of the distribution given by  softmax(𝑜)  and show that it matches the second derivative computed above.**
    
    For any exponential distribution family function, the first derivative of the cumulant function is mean, while second derivative is the variance of the corresponding distribution.
    
    ![title](image/textbook_solution_4.4.3.png)
    
<img src="image/textbook_solution_4.4.3.png" alt="Drawing" style="width: 20px;"/>


4. **Assume that we three classes which occur with equal probability, i.e. the probability vector is  ($\frac{1}{3}$, $\frac{1}{3}$, $\frac{1}{3}$).**

    a. **What is the problem if we try to design a binary code for it? Can we match the entropy lower bound on the number of bits?**
    
    `H(p) = 3*np.log2(3) = np.log2(27) ~= 4.75488`, However, we will need 5 bits to encode.
    
    b. **Can you design a better code. Hint - what happens if we try to encode two independent observations? What if we encode  𝑛  observations jointly?**
    
    
    
5. **Softmax is a misnomer for the mapping introduced above (but everyone in deep learning uses it). The real softmax is defined as $\text{RealSoftMax(𝑎,𝑏)}=log(exp(𝑎)+exp(𝑏))$.**

    a. **Prove that  RealSoftMax(𝑎,𝑏)>max(𝑎,𝑏).**
    
    $$RealSoftMax(𝑎,𝑏) = log(exp(𝑎)+exp(𝑏)) > log(exp(max(a,b))) = max(𝑎,𝑏)$$
    
    b. **Prove that this holds for  $𝜆^{−1} \text{RealSoftMax}(𝜆𝑎,𝜆𝑏)$ , provided that 𝜆>0.**

    $$𝜆^{−1} \text{RealSoftMax}(𝜆𝑎,𝜆𝑏) = 𝜆^{−1} log(exp(𝜆𝑎)+exp(𝜆𝑏)) > 𝜆^{−1} log(exp(max(𝜆𝑎,𝜆𝑏))) = max(a,b)$$
    
    c. **Show that for  𝜆→∞  we have  $𝜆^{−1} \text{RealSoftMax}(𝜆𝑎,𝜆𝑏)→max(𝑎,𝑏)$**.
    
    By Taylor expansion, $log(X+a) \simeq log(X) + a/X$, 
    
    Suppose $max(𝑎,𝑏) = a$, then,
    
    $$ 𝜆^{−1} log(exp(𝜆𝑎)+exp(𝜆𝑏)) \simeq 𝜆^{−1} log(exp(𝜆a)) + \frac{exp(𝜆𝑏)}{exp(𝜆a)} \simeq 𝜆^{−1} log(exp(𝜆a)) = max(𝑎,𝑏)$$
    
    d. **What does the soft-min look like?**
    
    For $\alpha < 0, softmin_{\alpha}(x_k) → min{x_k}$ as $\alpha → -∞$, where 
    
    $$softmin_{\alpha}(x_k) = \frac{\sum_k x_k \exp(\alpha x_k)}{\sum_k \exp(\alpha x_k)}$$
    
    e. **Extend this to more than two numbers.**
    
    As above.

#### 3.5.4. Exercises

1. **Does reducing batch_size (for instance, to 1) affect performance?**

    Update at each gradient descent (very noisy and may go off in a direction far from the minibatch gradient. However, the noisiness is exactly what you want in non-convex optimization, because it helps you escape from saddle points or local minimal.)
    
    
2. **For non-Windows users, try modifying num_workers to see how it affects read performance.**

    num_workers (default 0) is the number of multiprocessing workers to use for data preprocessing.
    
    
3. **Use the MXNet documentation to see which other datasets are available in mxnet.gluon.data.vision.**

    https://mxnet.incubator.apache.org/api/python/gluon/data.html#vision
    
4. **Use the MXNet documentation to see which other transformations are available in mxnet.gluon.data.vision.transforms.**

    https://mxnet.incubator.apache.org/api/python/gluon/data.html#vision-transforms


#### 3.6.9. Exercises

1. **In this section, we directly implemented the softmax function based on the mathematical definition of the softmax operation. What problems might this cause (hint - try to calculate the size of  exp(50) )?**

    exp(50) may lead to numerical overflow (when an arithmetic operation attempts to create a numeric value that is outside of the range that can be represented with a given number of digits).
    
    

2. **The function cross_entropy in this section is implemented according to the definition of the cross-entropy loss function. What could be the problem with this implementation (hint - consider the domain of the logarithm)?**

    The logarithm assume a positive input domain, hence we need to make sure the $\hat{y}$ is a strictly postive vector.

3. **What solutions you can think of to fix the two problems above?**

    We can reimplement the formula as:
    $$\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(X_{ij} - b )  \exp(b)}{\sum_k \exp(X_{ik} - b)  \exp(b)} = \frac{ \exp(X_{ij} - b) }{ \sum_{k=1}^n \exp(X_{ik} - b) },$$ where, $b = \max_{k=1}^n x_k$.


4. **Is it always a good idea to return the most likely label. E.g. would you do this for medical diagnosis?**

    No. For dataset with extremely unbalanced classes, if we evaluate the model accuracy by having same penalty on each sample (rather than each class), the model may turn to return the most likely label all the time. For example, in medical diagnosis of rear disease, we have to punish more on wrong prediction of potisive (disease) class.
    

5. **Assume that we want to use softmax regression to predict the next word based on some features. What are some problems that might arise from a large vocabulary?**

    For a large vocabulary, the choose space of next word will be large, and hence the matrix will be sparse.


#### 3.7.5. Exercises

1. **Try adjusting the hyper-parameters, such as batch size, epoch, and learning rate, to see what the results are.**

    Larger batch size => Lack of ability to generalize => "saddle point"
    
    Larger epoch => The more times that the model goes through the dataset => best accuracy in general
    
    Learning rate(lr):
    
    a. If lr too large, the model may oscillate around optimal point, or even get diverged in the end. 
    b. If lr too small, it may take too long to train. Even though the loss and train accuracy continues improving at a slow pace, since the gradient is too small at each step.


2. **Why might the test accuracy decrease again after a while? How could we fix this?**

    If the model get overfitted with training set, it may not be able to generalize to test set. 
